In [1]:
import numpy as np
import sympy as sp
from sympy import init_printing
from sympy import Matrix
from sympy.interactive import printing
printing.init_printing(use_latex=True)
sin=sp.sin
cos=sp.cos
pi=sp.pi

In [2]:
#Funciones de rotación canonicas con valores simbolicos
def srotx(ang):
    Rx=sp.Matrix([[1,0,0],
                  [0,sp.cos(ang),-sp.sin(ang)],
                  [0, sp.sin(ang), sp.cos(ang)]])
    return Rx
def sroty(ang):
    Ry= sp.Matrix([[sp.cos(ang),0,sp.sin(ang)],
                  [0,1,0],
                  [-sp.sin(ang), 0, sp.cos(ang)]])
    return Ry
def srotz(ang):
    Rz= sp.Matrix([[sp.cos(ang),-sp.sin(ang),0],
                  [sp.sin(ang), sp.cos(ang), 0],
                  [0,0,1]])
    return Rz


#T given t,R
def T_tR(t,R):
    T = np.vstack((np.hstack((R,t)),np.array([[0, 0, 0, 1]])))
    return T
def rotdh(d,th,a,alf):
    T=sp.Matrix([[cos(th),-cos(alf)*sin(th),sin(alf)*sin(th),a*cos(th)],
                  [sin(th),cos(alf)*cos(th),-sin(alf)*cos(th),a*sin(th)],
                  [0, sin(alf), cos(alf),d],[0, 0, 0, 1]])
    return T

In [164]:
m1, m2, i1, i2, lc1, lc2, q1, q2, l1 = sp.symbols('m1 m2 Izz1 Izz2 lc1 lc2 q1 q2 l1')

# Posiciones de los centros de masa
x1=lc1*cos(q1)
y1=lc1*sin(q1)
x2=l1*cos(q1)+lc2*cos(q1+q2)
y2=l1*sin(q1)+lc2*sin(q1+q2)
z1=1
z2=1
# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1
Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, 0],[0,0],[1,1]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R
R11=srotz(q1)
R12=srotz(q1+q2)
Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Ixy1 Iyy1 Iyz1 Ixz1 Iyz1 Izz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Ixy2 Iyy2 Iyz2 Ixz2 Iyz2 Izz2')

I1=sp.Matrix([[Ixx1, Ixy1, Ixz1],
             [Ixy1, Iyy1, Iyz1],
             [Ixz1, Iyz1, Izz1]])
I2=sp.Matrix([[Ixx2, Ixy2, Ixz2],
             [Ixy2, Iyy2, Iyz2],
             [Ixz2, Iyz2, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=m1*J1j1+m2*J2j2+Icdm1+Icdm2
Mq


#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])


g0 = sp.Matrix([[0, -g, 0]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T


def modelo_dinamico(Mq,C,g_v):
    return Mq, C, g_v

MD= modelo_dinamico(Mq,C,g_v)

In [173]:
# Ejemplo 2
m1, m2, q1, q2, l1, l2 = sp.symbols('m1 m2 q1 q2 l1 l2')

R11=sp.Matrix([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
R12=srotx(-pi/2)

x1=0
y1=0
z1=q1
x2=0
y2=q2
z2=q1
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[0,0]])
J2w=sp.Matrix([[0, 0],[0,0],[0,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Ixy1 Iyy1 Iyz1 Ixz1 Iyz1 Izz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Ixy2 Iyy2 Iyz2 Ixz2 Iyz2 Izz2')

I1=sp.Matrix([[Ixx1, Ixy1, Ixz1],
             [Ixy1, Iyy1, Iyz1],
             [Ixz1, Iyz1, Izz1]])
I2=sp.Matrix([[Ixx2, Ixy2, Ixz2],
             [Ixy2, Iyy2, Iyz2],
             [Ixz2, Iyz2, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=m1*J1j1+m2*J2j2+Icdm1+Icdm2
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,0 , -g]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T


def modelo_dinamico(Mq,C,g_v):
    return Mq, C, g_v

MD2= modelo_dinamico(Mq,C,g_v)


In [192]:
# Ejercicio 1
m1, m2, l1, l2, d1, d2, q1, q2 = sp.symbols('m1 m2 l1 l2 d1 d2 q1 q2')

R11=srotz(q1)
R12=sp.Matrix([[0,0,0],[0,0,0],[0,0,0]])

x1=d1*cos(q1)
y1=d1*sin(q1)
z1=1
# x2=l1*cos(q1)-((d2-q2)*cos(pi/2-q1))
# y2=l1*sin(q1)+((d2-q2)*sin(pi/2-q1))
x2=l1*cos(q1)-((q2)*cos(pi/2-q1))
y2=l1*sin(q1)+((q2)*sin(pi/2-q1))
z2=1
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, 0],[0,0],[1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Ixy1 Iyy1 Iyz1 Ixz1 Iyz1 Izz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Ixy2 Iyy2 Iyz2 Ixz2 Iyz2 Izz2')

I1=sp.Matrix([[Ixx1, Ixy1, Ixz1],
             [Ixy1, Iyy1, Iyz1],
             [Ixz1, Iyz1, Izz1]])
I2=sp.Matrix([[Ixx2, Ixy2, Ixz2],
             [Ixy2, Iyy2, Iyz2],
             [Ixz2, Iyz2, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=sp.simplify(m1*J1j1+m2*J2j2+Icdm1+Icdm2)
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[g,0 , 0]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T



In [72]:
Jv2, Jv1,g_v,g0

⎛                  ⎡-l₁⋅sin(q₁)    ⎤                                    ⎞
⎜                  ⎢────────────  0⎥                                    ⎟
⎜⎡-l₁⋅sin(q₁)  0⎤  ⎢     2         ⎥  ⎡⎡g⋅l₁⋅(m₁ + 2⋅m₂)⋅cos(q₁)⎤⎤  ⎡0 ⎤⎟
⎜⎢              ⎥  ⎢               ⎥  ⎢⎢────────────────────────⎥⎥  ⎢  ⎥⎟
⎜⎢l₁⋅cos(q₁)   0⎥, ⎢ l₁⋅cos(q₁)    ⎥, ⎢⎣           2            ⎦⎥, ⎢-g⎥⎟
⎜⎢              ⎥  ⎢ ──────────   0⎥  ⎢                          ⎥  ⎢  ⎥⎟
⎜⎣     0       1⎦  ⎢     2         ⎥  ⎣           [0]            ⎦  ⎣0 ⎦⎟
⎜                  ⎢               ⎥                                    ⎟
⎝                  ⎣     0        0⎦                                    ⎠

In [71]:
#Ejercicio 2 

m1, m2, l1, d2, q1, q2 = sp.symbols('m1 m2 l1 d2 q1 q2')

R11=srotz(q1)
R12=sp.Matrix([[1,0,0],[0,1,0],[0,0,1]])

x1=l1*cos(q1)/2
y1=l1*sin(q1)/2
z1=0

x2=l1*cos(q1)
y2=l1*sin(q1)
z2=q2

xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, 0],[0,0],[1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Ixy1 Iyy1 Iyz1 Ixz1 Iyz1 Izz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Ixy2 Iyy2 Iyz2 Ixz2 Iyz2 Izz2')

I1=sp.Matrix([[Ixx1,0, 0],
             [0, Iyy1, 0],
             [0, 0, Izz1]])
I2=sp.Matrix([[Ixx2, 0, 0],
             [0, Iyy2,0],
             [0, 0, Izz2]])


Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=sp.simplify(m1*J1j1+m2*J2j2+Icdm1+Icdm2)
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,-g , 0]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T


In [221]:
#Examen 3 ( 2017-1)

m1, m2, l1, q1, q2, d = sp.symbols('m1 m2 l1 q1 q2 d')


R11=srotz(q1)
R12=sp.Matrix([[0,0,0],[0,0,0],[0,0,0]])

x1=l1*cos(q1)/2
y1=l1*sin(q1)/2
z1=1

x2=l1*cos(q1)+(q2-d)*cos(pi/4+q1)
y2=l1*sin(q1)+(q2-d)*sin(pi/4+q1)
z2=1

xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, 0],[0,0],[1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Ixy1 Iyy1 Iyz1 Ixz1 Iyz1 Izz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Ixy2 Iyy2 Iyz2 Ixz2 Iyz2 Izz2')

I1=sp.Matrix([[Ixx1, Ixy1, Ixz1],
             [Ixy1, Iyy1, Iyz1],
             [Ixz1, Iyz1, Izz1]])
I2=sp.Matrix([[Ixx2, Ixy2, Ixz2],
             [Ixy2, Iyy2, Iyz2],
             [Ixz2, Iyz2, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=sp.simplify(m1*J1j1+m2*J2j2+Icdm1+Icdm2)
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,-g , 0]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T





In [75]:
#PC4 (2018-1)

m1, m2, l1, d2, q1, q2, d1 = sp.symbols('m1 m2 d1 d2 q1 q2 d1')


R11=srotz(pi/2+q1)*srotx(pi/2)
R12=srotz(q1+pi/2)*srotx(pi/2)*srotz(q2)

x1=0
y1=0
z1=d1

x2=d2*cos(q2)*sin(pi/2+q1)
y2=d2*cos(q2)*cos(pi/2+q1)
z2=d2*sin(q2)+l1


xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, cos(q1)],[0,sin(q1)],[1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ix1 Ixy1 Ixz1 Ixy1 Iy1 Iyz1 Ixz1 Iyz1 Iz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ix2 Ixy2 Ixz2 Ixy2 Iy2 Iyz2 Ixz2 Iyz2 Iz2')

I1=sp.Matrix([[Ixx1, 0, 0],
             [0, Iyy1, 0],
             [0,0, Izz1]])
I2=sp.Matrix([[Ixx2,0, 0],
             [0, Iyy2, 0],
             [0, 0, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=sp.simplify(m1*J1j1+m2*J2j2+Icdm1+Icdm2)
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols( \dot{q}_1 \dot{q}_2, q2\',g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,0 , -g]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T


#sp.simplify(Mq[0,0]-(Iyy1+Iyy2+m2*(d2**2)+(Ixx2-Iyy2-m2*d2**2)*sin(q2)**2)) 


In [121]:
#PC3 (2016-1)

m1, m2, d1, d2, q1, q2 = sp.symbols('m1 m2 d1 d2 q1 q2')


R11=sp.Matrix([[1, 0, 0],[0, 1, 0],[0,0,1]])
R12=sroty(q2)

x1=0
y1=0
z1=d1/2-q1

x2=-d2*cos(pi/2-q2)
y2=0
z2=d1+d2*sin(pi/2-q2)


xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv2=sp.Matrix.hstack(Jv21,Jv22)
Jv2

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])
J2w=sp.Matrix([[0, cos(q1)],[0,sin(q1)],[1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)


#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ix1 Ixy1 Ixz1 Ixy1 Iy1 Iyz1 Ixz1 Iyz1 Iz1')
Ixx2, Ixy2, Ixz2, Ixy2, Iyy2, Iyz2, Ixz2, Iyz2, Izz2 = sp.symbols('Ix2 Ixy2 Ixz2 Ixy2 Iy2 Iyz2 Ixz2 Iyz2 Iz2')

I1=sp.Matrix([[Ixx1, 0, 0],
             [0, Iyy1, 0],
             [0,0, Izz1]])
I2=sp.Matrix([[Ixx2,0, 0],
             [0, Iyy2, 0],
             [0, 0, Izz2]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm1, Icdm2

Mq=sp.simplify(m1*J1j1+m2*J2j2+Icdm1+Icdm2)
Mq



#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,0 , -g]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0 + Jv2[:,0].T*m2*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0 + Jv2[:,1].T*m2*g0))
g_v = sp.Matrix([[g1,g2]]).T


In [122]:
Mq

⎡           2                    2                           (-Ix₂ + Iz₂)⋅(-si
⎢    Ix₂⋅sin (q₂) + Iz₁ + Iz₂⋅cos (q₂) + m₁                  ─────────────────
⎢                                                                             
⎢                                                                             
⎢(-Ix₂ + Iz₂)⋅(-sin(q₁ - 2⋅q₂) + sin(q₁ + 2⋅q₂))         2        2           
⎢───────────────────────────────────────────────  Ix₂⋅cos (q₁)⋅cos (q₂) + Iy₂⋅
⎣                       4                                                     

n(q₁ - 2⋅q₂) + sin(q₁ + 2⋅q₂))           ⎤
──────────────────────────────           ⎥
      4                                  ⎥
                                         ⎥
   2              2        2         2   ⎥
sin (q₁) + Iz₂⋅sin (q₂)⋅cos (q₁) + d₂ ⋅m₂⎥
                                         ⎦

In [106]:
#Examen final

a1, a2, a3, q2 = sp.symbols('a1 a2 a3 q2')

Mq=sp.Matrix([[a1, a2*sin(q2)],[a2*sin(q2), a3]])


#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)
q=sp.Matrix([q1,q2])

# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])


sp.diff(Mq,q2)-2*C


⎡0  \dot{q}₂⋅a₂⋅cos(q₂)⎤
⎢                      ⎥
⎣0           0         ⎦

In [118]:
#Examen Final
q1, q2, dc2, m2 = sp.symbols('q1 q2 dc2 m2')
x1=(q2-dc2)*cos(q1)
y1=(q2-dc2)*sin(q1)
z1=1

R11=srotz(q1)

xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv1=sp.Matrix.hstack(Jv11,Jv12)
Jv1

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0],[0,0],[1,0]])


#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)



#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Ixy1, Iyy1, Iyz1, Ixz1, Iyz1, Izz1 = sp.symbols('Ix Ixy1 Ixz1 Ixy1 Iy Iyz1 Ixz1 Iyz1 Iz')

I1=sp.Matrix([[Ixx1, 0, 0],
             [0, Iyy1, 0],
             [0,0, Izz1]])

Icdm1=Jw1R1*I1*Jw1R1.T

Icdm1

Mq=sp.simplify(m1*J1j1+Icdm1)
Mq




#Calculo de Matriz Coriolisis/ fuerzas centrifugas
dq1, dq2,g=sp.symbols('\dot{q}_1 \dot{q}_2,g')

#Cij=sum cijk*qk (k=1 --> k=2)
#Cijk=1/2(dmij/dqk + dmik/dqj - dmjk/dqi)


# Reemplazando los terminos
#c111
mij11=sp.diff(Mq[0,0],q1) #mij
mik11=sp.diff(Mq[0,0],q1) #mik
mjk11=sp.diff(Mq[0,0],q1) #mjk
c111=(mij11+mik11-mjk11)/2

#c112 
mij11=sp.diff(Mq[0,0],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q1) #mik derivada respecto a jTHE 
mjk12=sp.diff(Mq[0,1],q1) #mjk derivada respecto a i
c112=(mij11+mik11-mjk11)/2

#c121
c121=c112

#c122 Mq[i-1,j-1] 
mij12=sp.diff(Mq[0,1],q2) #mij derivada respecto a k
mik12=sp.diff(Mq[0,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q1) #mjk derivada respecto a i
c122=(mij12+mik12-mjk22)/2

#c211 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q1) #mij derivada respecto a k
mik21=sp.diff(Mq[1,0],q1) #mik derivada respecto a j
mjk11=sp.diff(Mq[0,0],q2) #mjk derivada respecto a i
c211=(mij21+mik21-mjk11)/2

#c212 Mq[i-1,j-1]
mij21=sp.diff(Mq[1,0],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q1) #mik derivada respecto a j
mjk12=sp.diff(Mq[0,1],q2) #mjk derivada respecto a i
c212=(mij21+mik22-mjk12)/2

#c221
c221=c212

#c222 Mq[i-1,j-1]
mij22=sp.diff(Mq[1,1],q2) #mij derivada respecto a k
mik22=sp.diff(Mq[1,1],q2) #mik derivada respecto a j
mjk22=sp.diff(Mq[1,1],q2) #mjk derivada respecto a i
c222=(mij22+mik22-mjk22)/2

c11=c111*dq1+c112*dq2
c12=c121*dq1+c122*dq2
c21=c211*dq1+c212*dq2
c22=c221*dq1+c222*dq2

# Matriz Coriolis / Fuerzas centrifugas

C=sp.Matrix([[c11, c12],[c21, c22]])
C


#Vector de gravedad
g0 = sp.Matrix([[0,0 , -g]]).T
g1 = sp.simplify(-(Jv1[:,0].T*m1*g0))
g2 = sp.simplify(-(Jv1[:,1].T*m1*g0))
g_v = sp.Matrix([[g1,g2]]).T


In [144]:
#Examen final pregunta 1

q1, q2, q3, d1,d2, l2, m1, m2, m3 = sp.symbols('q1 q2 q3 d1 d2 l2 m1 m2 m3')

x1=0
y1=q1
z1=0

x2=d2*cos(q2)
y2=d1+q1+d2*sin(q2)
z2=0

x3=l2*cos(q2)-q3*cos(pi/2-q2)
y3=q1+d1+l2*sin(q2)+q3*sin(pi/2-q2)
z3=0



R11=sp.Matrix([[1, 0, 0],[0, 1, 0],[0,0,1]])
R12=srotz(q2)
R13=srotz(q2)


xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
xyz3=sp.Matrix([[x3],[y3],[z3]])

# Jacobianos de velocidad lineal (del CdM)
xyz1=sp.Matrix([[x1],[y1],[z1]])
xyz2=sp.Matrix([[x2],[y2],[z2]])
xyz3=sp.Matrix([[x3],[y3],[z3]])

Jv11=sp.diff(xyz1,q1)
Jv12=sp.diff(xyz1,q2)
Jv13=sp.diff(xyz1,q3)
Jv1=sp.Matrix.hstack(Jv11,Jv12,Jv13)
Jv1

Jv21=sp.diff(xyz2,q1)
Jv22=sp.diff(xyz2,q2)
Jv23=sp.diff(xyz2,q3)
Jv2=sp.Matrix.hstack(Jv21,Jv22,Jv23)
Jv2

Jv31=sp.diff(xyz3,q1)
Jv32=sp.diff(xyz3,q2)
Jv33=sp.diff(xyz3,q3)
Jv3=sp.Matrix.hstack(Jv31,Jv32,Jv33)
Jv3

#Jacobianos de velocidad angular
J1w=sp.Matrix([[0, 0,0],[0,0,0],[0,0,0]])
J2w=sp.Matrix([[0, 0,0],[0,0,0],[0,1,0]])
J3w=sp.Matrix([[0, 0,0],[0,0,0],[0,1,0]])

#Productos de Jacobianos de velocidad lineal
J1j1=sp.simplify(Jv1.T*Jv1)
J2j2=sp.simplify(Jv2.T*Jv2)
J3j3=sp.simplify(Jv3.T*Jv3)

#Productos de Jacobianos de velocidad angular con R

Jw1R1=J1w.T*R11
Jw2R2=J2w.T*R12
Jw3R3=J3w.T*R13

#Productos con el tensor de inercia de cada CdM
Ixx1, Ixy1, Ixz1, Iyx1, Iyy1, Iyz1, Izx1, Izy1, Izz1 = sp.symbols('Ixx1 Ixy1 Ixz1 Iyx1 Iyy1 Iyz1 Izx1 Izy1 Izz1')
Ixx2, Ixy2, Ixz2, Iyx2, Iyy2, Iyz2, Izx2, Izy2, Izz2 = sp.symbols('Ixx2 Ixy2 Ixz2 Iyx2 Iyy2 Iyz2 Izx2 Izy2 Izz2')
Ixx3, Ixy3, Ixz3, Iyx3, Iyy3, Iyz3, Izx3, Izy3, Izz3 = sp.symbols('Ixx3 Ixy3 Ixz3 Iyx3 Iyy3 Iyz3 Izx3 Izy3 Izz3')

I1=sp.Matrix([[Ixx1, Ixy1, Ixz1],
             [Iyx1, Iyy1, Iyz1],
             [Izx1,Izy1, Izz1]])
I2=sp.Matrix([[Ixx2, Ixy2, Ixz2],
             [Iyx2, Iyy2, Iyz2],
             [Izx2,Izy2, Izz2]])
I3=sp.Matrix([[Ixx3, Ixy3, Ixz3],
             [Iyx3, Iyy3, Iyz3],
             [Izx3,Izy3, Izz3]])

Icdm1=Jw1R1*I1*Jw1R1.T
Icdm2=Jw2R2*I2*Jw2R2.T
Icdm3=Jw3R3*I3*Jw3R3.T
Icdm1, Icdm2, Icdm3

Mq=sp.simplify(m1*J1j1+m2*J2j2+m3*J3j3+Icdm1+Icdm2+Icdm3)
Mq







⎡                m₁ + m₂ + m₃                  d₂⋅m₂⋅cos(q₂) + m₃⋅(l₂⋅cos(q₂) 
⎢                                                                             
⎢                                                                 2         ⎛ 
⎢d₂⋅m₂⋅cos(q₂) + m₃⋅(l₂⋅cos(q₂) - q₃⋅sin(q₂))     Izz₂ + Izz₃ + d₂ ⋅m₂ + m₃⋅⎝l
⎢                                                                             
⎣                 m₃⋅cos(q₂)                                      l₂⋅m₃       

- q₃⋅sin(q₂))  m₃⋅cos(q₂)⎤
                         ⎥
 2     2⎞                ⎥
₂  + q₃ ⎠        l₂⋅m₃   ⎥
                         ⎥
                   m₃    ⎦

In [141]:
Jv1,Jv2,Jv3,J1w,J2w,J3w

⎛⎡0  0  0⎤  ⎡0  -d₂⋅sin(q₂)  0⎤  ⎡0  -d₂⋅sin(q₂) - q₃⋅cos(q₂)  -sin(q₂)⎤  ⎡0  
⎜⎢       ⎥  ⎢                 ⎥  ⎢                                     ⎥  ⎢   
⎜⎢1  0  0⎥, ⎢1  d₂⋅cos(q₂)   0⎥, ⎢0        -q₃⋅sin(q₂)         cos(q₂) ⎥, ⎢0  
⎜⎢       ⎥  ⎢                 ⎥  ⎢                                     ⎥  ⎢   
⎝⎣0  0  0⎦  ⎣0       0       0⎦  ⎣0             0                 0    ⎦  ⎣0  

0  0⎤  ⎡0  0  0⎤  ⎡0  0  0⎤⎞
    ⎥  ⎢       ⎥  ⎢       ⎥⎟
0  0⎥, ⎢0  0  0⎥, ⎢0  0  0⎥⎟
    ⎥  ⎢       ⎥  ⎢       ⎥⎟
0  0⎦  ⎣0  1  0⎦  ⎣0  1  0⎦⎠

In [146]:
display(Mq)

⎡                m₁ + m₂ + m₃                  d₂⋅m₂⋅cos(q₂) + m₃⋅(l₂⋅cos(q₂) 
⎢                                                                             
⎢                                                                 2         ⎛ 
⎢d₂⋅m₂⋅cos(q₂) + m₃⋅(l₂⋅cos(q₂) - q₃⋅sin(q₂))     Izz₂ + Izz₃ + d₂ ⋅m₂ + m₃⋅⎝l
⎢                                                                             
⎣                 m₃⋅cos(q₂)                                      l₂⋅m₃       

- q₃⋅sin(q₂))  m₃⋅cos(q₂)⎤
                         ⎥
 2     2⎞                ⎥
₂  + q₃ ⎠        l₂⋅m₃   ⎥
                         ⎥
                   m₃    ⎦